# Coordle Idea
1. Get query, e.g. "What is love?"
2. Tokenize query
3. Create a vector using word2vec of the tokens by summing them, or finding the average or whatever.
   lets call it the query vector. 
4. Find sentence vectors that are the closesest to the query vector.
5. Return the papers corresponding to said sentence vectors.

# Coordle Frontend
💩

# Coordle Backend

In [1]:
import pandas as pd 
import numpy as np
from importlib import reload
import nltk
import spacy
import en_core_sci_lg # Biomedical word embeddings
from utils import clean_text, load_pickle, save_pickle
from nltk.corpus import stopwords as _stopwords
from tqdm import tqdm
from collections import deque
from copy import deepcopy
from collections.abc import Iterable
from typing import Union
from pprint import pprint
from itertools import chain

import os
import json
import re 
from string import punctuation as PUNCTUATION
from nltk.corpus import stopwords as _stopwords
from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

In [2]:
df = pd.read_csv('data/cord-19-data.csv')

In [3]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, logs_filename: str):
        self.output_dir = output_dir
        self.prefix = prefix
        self.logs_filename = logs_filename
        self.epoch = 0

    def on_epoch_end(self, model):
        cum_loss = model.get_latest_training_loss()
        if self.epoch == 0:
            loss = cum_loss
        else:
            loss = cum_loss - self.loss_previous_step
        self.loss_previous_step = loss
        with open(join_path(self.output_dir, self.logs_filename), 'a+') as file:
            file.write(f'Epoch #{self.epoch}, loss: {loss}\n')
        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self, output_dir: str, prefix: str, start_epoch: int = 1):
        self.output_dir = output_dir
        self.prefix = prefix
        self.epoch = start_epoch

    def on_epoch_end(self, model):        
        output_path = join_path(self.output_dir, f'{self.prefix}_epoch_{self.epoch}.model')
        model.save(output_path)
        self.epoch += 1    

In [4]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec', 'w2v_model_epoch_29.model'))
word_to_int = {word:i for i, word in enumerate(model.wv.index2word)}
int_to_word = np.array(model.wv.index2word)

In [5]:
# %%time
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser
Index = Index2

def test_CordDoc():
    docsample = df[df.cord_uid == 'vs5yondw']
#     doc = CordDoc(docsample['cord_uid'], docsample['title'])
#     doc, _ = doc.fit(docsample['body_text'])
    gaga = Index()
    gaga.build_from_df(
        docsample,
#         df[:32],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=False,
        workers=-1
    )
    
    return gaga
#     return doc

def test_Index():
    coordle = Index()
    coordle.build_from_df(
        df.iloc[:4196],
        'cord_uid',
        'title',
        'body_text', 
        verbose=True, 
        use_multiprocessing=True,
        workers=-1
    )
    return coordle

index = test_Index()
# fuck = test_CordDoc()

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 4196/4196 [00:12<00:00, 335.01it/s]


In [6]:
# save_pickle(index, 'coordle/coordle_index_8k_0904.p')

In [7]:
query = 'retarded white woman'
docs1, score, errmsgs = index.search(query, verbose=True)
print(docs1)
print()
# docs2 = index.advanced_search(query, verbose=True)
# print(docs2)

[ytzglkgk, nhrzc7tj, up7hlqck, xsjdy3yz, tloaa3v1, ggsstwcm, 3gnfbgeo, dt0jku7v, 5ulk3euw, 4r61gaw2, aza0pzud, ke1r2e1b, dbu93ufh, yhunwsdq, qav5okgk, wisaamn7, ragy7afs, sbixelq5, z5wjge0x, r8j6lhoc, ieex2xwx, 5is9kc52, qg2r691d, 4nhy35xn, d28li5ao, pad3x324, epylkxdx, cd8u42xo, l7kaj498, cxth3hyn, ab63ijxx, bcd1q5an, bnnl700a, cgks5ap7, lahy0jfx, zroskqmk, tyid60aa, u5qd9lcd, ipvrc5we, qkedjzqs, p4exyu56, n1kkcl1w, b18hnqsl, yci0a6bt, mg2zikyp, bb27z7h0, 8mvsqh6t, dkp0fwe3, 6eer4qgr, 4qfhltg4, a9wmor1i, niurdu7t, 3dpnmswf, uniz8tuc, f7ky61ds, gcjgfasj, dh9n3j23, pl6gd3w4, qatiqnac, eb3asgvi, hwjkbpqp, u6strrib, 82m3w06s, 3xeg6kue, 06e9lkwl, dbmfydyu, m81xn0fs, zwfxnd7r, ffiguxo8, mbv7pk21, twvd7yke, lluqlzlg, d5d8amk7, 0r1b44id, kzv9cmr1, shnejtjt, 7uukeqea, 0ht2ap9g, d2covv62, 6anybxnk, qww6pe61, 73mhcnng, 8jsjyeio, bf2i20nq, v08d3oag, sxohecoi, ikx68zp0, 5bg7tswp, ufjjengt, eva0soja, ow2xqhmp, nr8fwc8q, lzquegwc, l7rn00vq, l42klpz6, 6rujx4vs, zq22sfna, dp6vzgef, sqe7h05t, p1ajw8mq,

In [8]:
query = 'retarded    (white AND woman)'
querytokens = re.split('([^a-zA-Z0-9])', query)
# Gotta do this to capture parenthesis
querytokens = chain.from_iterable([t.split() for t in querytokens])

print(list(querytokens))

['retarded', '(', 'white', 'AND', 'woman', ')']


In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2
AI_Index = AI_Index2

# ai_index = AI_Index(model.wv.most_similar, n_similars=1)
ai_index = Index()
ai_index.build_from_df(
    df.iloc[:16000],
    'cord_uid',
    'title',
    'body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)


Text cleaning initilized on 16 workers


Adding to index:  87%|████████▋ | 13901/16000 [00:44<00:08, 251.18it/s]

In [ ]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2
AI_Index = AI_Index2

docs, scores, errmsgs = ai_index.search('retarded AND woman')
n = 69
if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')


In [11]:
import coordle_backend
reload(coordle_backend)
from coordle_backend import CordDoc, Index2, RecursiveDescentParser, AI_Index2
Index = Index2

queries = [
    'retarded!!!',
#     '(',
#     ')',
    'retarded (white AND (woman NOT man))',
    'retarded (white AND white) man',
#     'retarded OR white OR woman',
#     'retarded white AND woman',
#     'retarded OR white NOT woman',
#     'retarded (white NOT woman)',
#     'retarded (white NOT woman)',
#     'OR retarded AND white woman',
#     'retarded AND AND white NOT woman',
#     'retarded (white NOT woman) AND',
#     ')retarded ((white NOT woman) AND',
#     'retarded ((white NOT woman)',
#     'AND retarded)) ((white NOT woman) NOT',
]

rdp = RecursiveDescentParser(index.docmap)
for query in queries:
    errmsgs = []
    tokens = rdp.get_logical_querytokens(query)
    pass_ = rdp.assert_query(tokens, errmsgs)
    
#     print(tokens)
    print(rdp.parenthesis_handler(tokens))
#     print(errmsgs)
    print(pass_)
    print()
    

deque(['retarded'])
True

deque(['retarded', 'OR', deque(['white', 'AND', deque(['woman', 'NOT', 'man'])])])
True

deque(['retarded', 'OR', deque(['white', 'AND', 'white']), 'OR', 'man'])
True



In [28]:
with open('textfile.txt', 'w+') as f:
#     uid = docs2_list[0].uid
#     uid = 'dlh93ax6'
#     uid = 'zp9k1k3z'
#     uid = 'vs5yondw'
    uid = 'qzm9wgde'
    
    print(f'dumping {uid}')
    f.write(df[df.cord_uid == uid].body_text.values[0].lower())

dumping qzm9wgde


In [ ]:
raise ValueError

## Multiprocessing showdown

In [ ]:
from multiprocessing import Pool
from time import time

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text), position=0, total=len(df)))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text), position=0, total=len(df))
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
with Pool(None) as p:
    cleaned_texts = list(tqdm(p.imap(clean_text, df.body_text[:2048]), position=0, total=2048))
print(f'Text cleaning with multiprocessing took {time()-t0:.2f} seconds')

In [ ]:
t0 = time()
gen = tqdm((clean_text(text) for text in df.body_text[:2048]), position=0, total=2048)
cleaned_texts = list(gen)
print(f'Naïve text cleaning took {time()-t0:.2f} seconds')